# 🧮 Inventory-Aware Promotion Strategy
*Balancing purchase intent with stock levels for smart campaign planning*

This notebook combines predicted **purchase probabilities** with simulated **inventory levels** to rank products for promotion.
The goal is to prioritize items that are both in high demand and well-stocked — avoiding overpromoting low-inventory items or underselling popular ones.

We also segment top-ranked products by **user type**, enabling personalized campaign targeting.

## 📦 Load Product Scores and Simulate Inventory Levels
We load the **predicted purchase probabilities** from the previous notebook and simulate **inventory availability** for each product.

This step enables us to:

- Prioritize in-stock, high-conversion items

- Avoid promoting out-of-stock products

In a real system, inventory data would be pulled from a database or API. Here, we simulate it using random values to mimic realistic stock levels.

In [1]:
import os
import pandas as pd
import numpy as np

# Paths Setup
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
PROCESSED_DIR = os.path.join(BASE_DIR, "data", "processed")

# Load predicted purchase probabilities
df = pd.read_csv(os.path.join(PROCESSED_DIR, "purchase_predictions.csv"))

# Simulate inventory levels (units in stock)
np.random.seed(42)
df["inventory"] = np.random.randint(0, 100, size=len(df))  # 0 = out of stock, 100 = high availability

df.head()

,product_id,purchase_prob,inventory
0,7198,0.094240,51
1,57335,0.061483,92
2,18474,0.259284,14
3,15681,0.145383,71
4,59557,0.089866,60


## 📊 Score Products by Promotion Priority

To ensure marketing efforts are both effective and sustainable, we score each product based on two key signals:

- **Purchase Likelihood** — from the machine learning model  
- **Inventory Availability** — current stock level

We combine them into a single score:

$$\text{promotion\_score} = \text{purchase\_prob} \times \text{inventory\_weight}$$

This allows us to **prioritize products that are both likely to convert and sufficiently stocked**, avoiding wasted budget on low-inventory items.

In [2]:
# Normalize inventory to [0, 1] scale
df['inventory_weight'] = df['inventory'] / df['inventory'].max()

# Compute final promotion score
df['promotion_score'] = df['purchase_prob'] * df['inventory_weight']

# Sort products by promotion score (highest priority first)
df_sorted = df.sort_values("promotion_score", ascending=False)
df_sorted.head(10)

,product_id,purchase_prob,inventory,inventory_weight,promotion_score
12639,34939,0.829996,97,0.979798,0.813229
5194,24455,0.820874,98,0.989899,0.812582
2958,43300,0.759061,98,0.989899,0.751394
16789,6663,0.807459,91,0.919192,0.742210
19911,50967,0.799194,90,0.909091,0.726540
10931,59213,0.733823,97,0.979798,0.718998
13277,48708,0.793570,89,0.898990,0.713412
19505,31462,0.782641,89,0.898990,0.703586
12808,12958,0.712684,96,0.969697,0.691088
5354,37520,0.800794,84,0.848485,0.679462


## 🧠 Segment Promotions by User Type

To support **personalized marketing**, we replicate the top-ranked products for each predefined user segment:

- **Budget**
- **Fashionista**
- **Color Lover**

Each segment receives the same top recommendations initially, but this step sets the foundation for future differentiation (e.g., ranking or filtering by user preferences).

In [6]:
user_types = ['budget', 'fashionista', 'color_lover']

# Create one campaign file per user segment
for segment in user_types:    
    segment_df = df_sorted.copy()
    segment_df['user_type'] = segment

    output_path = os.path.join(PROCESSED_DIR, f"campaign_{segment}.csv")
    segment_df.head(10).to_csv(output_path, index=False)

# Display the top 50 products globally
df_sorted.head(50)

,product_id,purchase_prob,inventory,inventory_weight,promotion_score
12639,34939,0.829996,97,0.979798,0.813229
5194,24455,0.820874,98,0.989899,0.812582
2958,43300,0.759061,98,0.989899,0.751394
16789,6663,0.807459,91,0.919192,0.742210
19911,50967,0.799194,90,0.909091,0.726540
10931,59213,0.733823,97,0.979798,0.718998
13277,48708,0.793570,89,0.898990,0.713412
19505,31462,0.782641,89,0.898990,0.703586
12808,12958,0.712684,96,0.969697,0.691088
5354,37520,0.800794,84,0.848485,0.679462


## 💾 Export Top Products for Deployment
We save the full ranked list and top 50 promoted products for use in deployment and campaign dashboards.

In [7]:
# Save full product ranking with promotion scores
df_sorted.to_csv(os.path.join(PROCESSED_DIR, "final_campaign_ranking.csv"), index=False)

# Save top 50 products with highest promotion scores
df_sorted.head(50).to_csv(os.path.join(PROCESSED_DIR, "top_promoted_products.csv"), index=False)

print("✅ Saved:")
print(f"- All rankings → {os.path.join(PROCESSED_DIR, 'final_campaign_ranking.csv')}")
print(f"- Top 50 picks → {os.path.join(PROCESSED_DIR, 'top_promoted_products.csv')}")

✅ Saved:
- All rankings → /Users/letitiachang/Desktop/Projects/omni-retail-ai/data/processed/final_campaign_ranking.csv
- Top 50 picks → /Users/letitiachang/Desktop/Projects/omni-retail-ai/data/processed/top_promoted_products.csv


## ✅ Summary

In this notebook, we combined predicted purchase intent with simulated inventory levels to prioritize products for promotion.

- Scored each product using:
`promotion_score = purchase_prob × normalized_inventory`

- Ranked all products based on this score to support demand-driven promotions.

- Segmented top recommendations for different user personas:
**budget shoppers**, **fashionistas**, and **color lovers**.

- Exported full and segmented product rankings for downstream marketing and dashboard use.

➡️ Next: visualize campaigns or integrate recommendations into OmniRetail AI's user-facing tools.